In [ ]:
import importlib

import numpy as np
import scipy
import scipy.stats
import matplotlib.pyplot as plt

import keras
from keras import backend as K
import train

import pandas as pd

keras.__version__

In [ ]:
importlib.reload(train)

In [ ]:
csvpath = "./mydata/"
image_path = "/mnt/g/gshare/carnd/fwd2/IMG/"
#lines = train.loadDriveLog(csvpath)
#angles = np.array([float(x[3]) for x in lines])

In [ ]:
df=pd.read_csv(csvpath + "driving_log.csv", header=None, names=["center", "left", "right", "angle", "throttle", "brake", "speed"])

In [ ]:
pos_entry = df[df.angle > 0.].iloc[10]
neg_entry = df[df.angle < -0.1].iloc[10]; neg_entry
zero_entry = df[df.angle == 0.].iloc[10]; zero_entry

dfsub = pd.DataFrame([pos_entry, neg_entry, zero_entry]); dfsub

In [ ]:
X_train = train.loadImagesPd(dfsub, image_path)
y_train = dfsub["angle"].values

print(X_train.shape)


In [ ]:
# import cv2
# filename=dfsub["center"][10].split('\\')[-1]
# img = train.loadImage(filename, image_path)
# plt.imshow(img)
# plt.show()

# img = cv2.imread(image_path + filename); img
# plt.imshow(img)
# plt.show()

# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# plt.imshow(img)
# plt.show()

# imgs = np.zeros((2,img.shape[0], img.shape[1], img.shape[2]))
# imgs[0,:,:,:] = img
# print(img)
# print(imgs[0])
# plt.imshow(imgs[0])
# plt.show()

In [ ]:
print("max:", np.max(X_train))
print("min:", np.min(X_train))

#import matplotlib.pyplot as plt
#from matplotlib.colors import hsv_to_rgb
get_ipython().magic('matplotlib inline')
# plt.figure(1, figsize=(15,15))
for i in range(3):
#     plt.subplot(3,1,i+1)
    idx = i
    plt.imshow(X_train[idx])
    plt.axis('off')
    plt.title('{}'.format(y_train[idx,]))
    plt.show()

In [ ]:
### Plot some statistics
print("mean:\t", np.mean(y_train))
print("median:\t", np.median(y_train))
print("std:\t", np.std(y_train))
print("mode:\t", scipy.stats.mode(y_train))

plt.hist(y_train, bins=100)
plt.show()

In [ ]:
### check the cropping
model = train.nvidiaModel()
get_1st_layer_output = K.function([model.layers[0].input],
                                  [model.layers[1].output])
layer_output = get_1st_layer_output([X_train])[0]
plt.imshow(layer_output[2]*0.5 - 0.5)

In [ ]:
history = train.train(model, X_train, y_train, epochs=30)

In [ ]:
y_predict = model.predict(X_train)
print(y_predict)
print(y_train)

In [ ]:
importlib.reload(train)
print(df.iloc[0:10]["angle"])

#i = np.array([0, 9, 5, 3,])
i = np.arange(10)
g = train.generate_data(df.iloc[i], image_path, batch_size=4)
x,y = next(g)
print(x.shape)
print(y)

In [ ]:
history=train.train_gen(model, df.iloc[i], image_path, batch_size=32, epochs=20)
# help(model.fit_generator)

In [ ]:
X_test = train.loadImagesPd(df.iloc[i], image_path)
y_predict = model.predict(X_test)
print(y_predict)
print(df.iloc[i]["angle"])